In [105]:
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os 
import keras
from scipy import stats
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras.models import load_model
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, GlobalAveragePooling1D, GlobalAveragePooling2D, Flatten, BatchNormalization, Dense
from keras.applications.inception_v3 import InceptionV3
from keras.constraints import maxnorm
from keras import optimizers
from keras.optimizers import Adam, SGD , RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

## Load Pretrained image classification models 

from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3


In [2]:
## Seed Everything 
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


# Uses data generators to get data 

In [44]:
## Prepare data

def prepare_data(train_path,val_path,test_path,batch_size=256): 
    
    ## Initializae values 

    rescale = 1./255
    target_size = (150, 150)
    class_mode = "categorical"

    train_datagen = ImageDataGenerator(
        rescale=rescale,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    train_generator = train_datagen.flow_from_directory(
        train,
        target_size=target_size,
        class_mode=class_mode,
        batch_size = batch_size,
        shuffle=True)
    print(batch_size)

    val_datagen = ImageDataGenerator(rescale=rescale)

    val_generator = val_datagen.flow_from_directory(
        val,
        target_size=target_size,
        batch_size = batch_size,
        class_mode= class_mode)


    test_datagen = ImageDataGenerator(rescale=rescale)

    test_generator = test_datagen.flow_from_directory(
        test,
        target_size=target_size,
        class_mode= class_mode)

    return train_generator,val_generator,test_generator

    

# Standard Keras callbacks

In [45]:
def setup_keras_callbacks(model_file,log_dir,batch_size=256): 
    checkpoint = ModelCheckpoint(
    model_file, 
    monitor='val_acc', 
    save_best_only=True)

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        verbose=1,
        restore_best_weights=True)


    tensorboard = TensorBoard(
        log_dir=log_dir,
        batch_size=batch_size,
        update_freq = 'batch')


    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        patience=5,
        cooldown=2,
        min_lr=0.0000000001,
        verbose=1)

    callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
    
    return callbacks


# Run this cell to train inception, vgg and resnet 

In [48]:
## Seed everything so that the outputs are deterministic 

seed_everything(42)
root_dir ='/Users/akalyana/DL/MO/AS4/chest_xray/'
train = root_dir + 'train/'
val  = root_dir + 'val/'
test  = root_dir + 'test/'
out = root_dir + 'output/'
# Prepare data

train_gen, val_gen, test_gen =  prepare_data(train,val,test,128)

print(len(train_gen), len(val_gen), len(test_gen))

#Get base models
base_models = [] 
base_model_type = ['inception','vgg19', 'resnet']
mod_models = [] 
for bt in base_model_type: 
    base_models.append(get_base_model(model_type=bt,input_shape=(150,150,3),freeze='all'))

#Add and retrain dense layers 

for model in base_models: 
    mod_models.append(add_last_layer(model))
#Setup callbacks 

model_file = root_dir + 'output/' +  "{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"
log_dir = root_dir + 'output/log/' 
callbacks = setup_keras_callbacks(model_file,log_dir,1024)

#Train models

epochs = 10

for model in mod_models: 
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(
        train_gen,
        steps_per_epoch = len(train_gen),
        epochs = epochs,
        validation_data = val_gen,
        validation_steps = len(val_gen),
        callbacks = callbacks)


Found 5216 images belonging to 2 classes.
128
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
41 5 1


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


Epoch 1/10
41/41 [==============================] - 263s 6s/step - loss: 0.4424 - acc: 0.8364 - val_loss: 0.7818 - val_acc: 0.7340
Epoch 2/10
41/41 [==============================] - 351s 9s/step - loss: 0.2367 - acc: 0.9012 - val_loss: 1.8212 - val_acc: 0.6795
Epoch 3/10
41/41 [==============================] - 257s 6s/step - loss: 0.2180 - acc: 0.9169 - val_loss: 1.2430 - val_acc: 0.7163
Epoch 4/10
41/41 [==============================] - 259s 6s/step - loss: 0.1927 - acc: 0.9223 - val_loss: 1.5389 - val_acc: 0.7115
Epoch 5/10
41/41 [==============================] - 263s 6s/step - loss: 0.2073 - acc: 0.9151 - val_loss: 0.9943 - val_acc: 0.7147
Epoch 6/10
41/41 [==============================] - 287s 7s/step - loss: 0.1931 - acc: 0.9243 - val_loss: 2.3506 - val_acc: 0.6635

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Restoring model weights from the end of the best epoch
Epoch 00006: early stopping
Epoch 1/10
41/41 [==============================]

# Add 2 fully connected layers at the end of Inception, VGG and Resnet

In [ ]:
def add_last_layer(base_model):
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
    x=Dense(512,activation='relu')(x) #dense layer 3
    preds=Dense(2,activation='softmax')(x) #final layer w
    model = Model(inputs=base_model.input,outputs=preds)
    return model

# Get base model

In [115]:
def get_base_model(model_type='inception',input_shape=(150,150,3),freeze='all'): 
    if(model_type == 'inception'): 
        base_model = InceptionV3(weights='imagenet',input_shape=(150,150,3), include_top=False)
    elif(model_type == 'vgg19'):
        base_model = VGG19(weights='imagenet',input_shape=(150,150,3), include_top=False)
    elif(model_type == 'resnet'):
        base_model = ResNet50(weights='imagenet',input_shape=(150,150,3), include_top=False)
    #Decide how much to freeze 
    if freeze == 'all': 
        ll = len(base_model.layers)
    else: 
        ll = int(freeze)
    for layer in base_model.layers[:ll]:
        layer.trainable = False
    return base_model

# Predict once the models are trained 

In [118]:
##Load Models and Ensemble to predict
##Graph Results 
def predict(path, data_gen, ensemble=True): 
    preds = list()
    mod_eval = list()
    if(ensemble==True): 
        for f in os.listdir(path):
            if f.endswith('.hdf5'):
                model = load_model(path+f)
                preds.append(model.predict_generator(data_gen, steps = len(data_gen)))
                mod_eval.append(model.evaluate_generator(data_gen, steps = len(data_gen)))
        np_preds = np.array(preds)
        # Voting mechanism for the models 
        outcomes,_ = stats.mode(np_preds)
  #              preds.append(model.predict(val_generator, steps= len(val_generator)) 
        
    else: 
        for f in os.listdir(path): 
            a=1
            #Parse the highest val accuracy model 
            #Make the prediction 
                
    return outcomes,preds,mod_eval


In [119]:
outcomes, preds,eval = predict(root_dir + 'output/',test_gen, True)